In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot

In [2]:
data = pd.read_excel("./data/1_train.xlsx")
data.head(5)

,主页ID,主页名称,分类,message,name,description,发文类型,y
0,155000000000000,纬来体育台官方粉丝团,电视频道,中华队长林智胜 三分砲！！！\n人如其名 真的「致胜」阿～,NaN,NaN,video,H
1,359000000000,公视粉丝团,电视频道,您所选择的电视频道，将会决定您的未来~\n遥控器的决定权，就在你手上！\n\n#看见更好的未来,2015公视电视募款 公视让你看见更好的未来,这是一个资讯爆炸的时代，你所选择的资讯将决定你的视野；你所选择的电视频道，将会决定你的未来 ...,video,L
2,359000000000,公视粉丝团,电视频道,好演员值得更多肯定和掌声！\n\n吴慷仁提到大概有六、七年资历的演员，所面临的最大困境，是赚...,公视粉丝团,NaN,link,L
3,190000000000,Mobile01,网路社群,大师兄 一发炸裂！！帅啊啊啊啊啊啊啊！！！,来自 Mobile01 贴文的相片,NaN,photo,H
4,359000000000,公视粉丝团,电视频道,「再不救，就来不及了！如果没有很强力的救，再三十年，你可能找不到鹿港的味道了！」\n\n「两...,【我们的岛】抢救鹿港,叶明兰、叶镇中 / 采访报导 「再不救，就来不及了！如果没有很强力的救，再三十年，你可能找不...,link,L


In [4]:
from Python.pandas.feature_selection import *
object_feature_helper(data)


---------------------------------------------
data contains [ 7829 ] records 
主页ID  is numeric column:
count    7.829000e+03
mean     2.764040e+14
std      4.093619e+14
min      6.282507e+10
25%      1.090000e+14
50%      1.260000e+14
75%      2.420000e+14
max      1.650000e+15
Name: 主页ID, dtype: float64
---------------------------------------------



主页名称  has [ 133 ] different values, please create features yourself!
please notice here are some seldom_indexes:  Index(['PTT网路电台/PTTradio', '我的自由年代', '疯神无双', 'NOWnews.com 今日新闻网',
       '沃草！（Watchout）', '老外看中国、老外看台湾 A Laowai's View of China & Taiwan',
       '长不大的爸爸', '十二夜', 'Running Man 台湾之家', '金马影展 TGHFF', '【玩命关头】电影粉丝专属页面',
       '联合报plus', '苹果娱乐新闻', '听见幸福', 'PanSci 科学新闻网', '新头壳newtalk',
       'udn.com 联合新闻网', 'ifilm/传影互动', 'KKBOX', '得利影视．go movie 情报站', '食尚玩家',
       '大纪元时报 - 台湾(The Epoch Times - Taiwan)', '妹妹', '楼下的房客', '威视电影',
       'MamaClub', '公视HD 高画质频道', '综艺大集合', 'PTT Gossiping 批踢踢八卦板',
       '想想论坛 Thinking Taiwan Forum', '

主页ID 提取 blacklist, whitelist, data_hl_score

blacklist 和 whitelist 不加入训练名单, 直接判定。
419000000000000 只有一个正例，也加入黑名单。
data_hl_score 只对有50条记录以上Id计算，少于50条的记为0。
data_hl_score = high_count / total_count

In [22]:
grp = data.pivot_table(index='主页ID', columns='y', values=['分类'], aggfunc='count').droplevel(0, axis=1)
grp = grp.fillna(0)
blacklist = grp[((grp['H'] <= 1) & (grp['L'] > 5))]
whitelist = grp[(grp['L'] == 0) & (grp['H'] > 1)]

In [39]:
data_remove_bw = data[(~data['主页ID'].isin(blacklist.index.values)) & (~data['主页ID'].isin(whitelist.index.values))]
data_remove_bw = data_remove_bw.reset_index()
data_remove_bw

,index,主页ID,主页名称,分类,message,name,description,发文类型,y
0,0,155000000000000,纬来体育台官方粉丝团,电视频道,中华队长林智胜 三分砲！！！\n人如其名 真的「致胜」阿～,NaN,NaN,video,H
1,3,190000000000,Mobile01,网路社群,大师兄 一发炸裂！！帅啊啊啊啊啊啊啊！！！,来自 Mobile01 贴文的相片,NaN,photo,H
2,5,242000000000000,ETtoday新闻云,新闻,爽啦～～～（ Supermm）\n\n#中华队 #古巴\nhttp://www.ettoda...,故意保送后林智胜怒轰3分炮 中华睽违9年击败古巴 | ET运动云 | ETtoday东森新闻云,世界12强棒球赛中华队林智胜，8局下在前一棒郭严文被故意保送成一、二垒有人，轰出左外野3分全...,link,H
3,6,190000000000,Mobile01,网路社群,4：1，中华战胜古巴！今晚是最棒的周末夜，谢谢中华队！请继续加油！,来自 Mobile01 贴文的相片,NaN,photo,H
4,7,109000000000000,Yahoo!奇摩新闻,新闻,一个字，爽！！！！\n\n#12强 #台古大战 #林智胜 #棒球 #三分砲 Lamigo M...,世界12强／林智胜怒轰3分砲！中华队赢古巴啦,今天(14日)「大师兄」林智胜单场被看扁2次...,link,H
...,...,...,...,...,...,...,...,...,...
6746,7824,283000000000,中天快点TV,电视频道,被镶上绿色宝石的番薯，好美！（土星人）\n\n\n---\n●CH52中天新闻On Air！...,从太空夜拍台湾美翻了！ 嘉南平原就像被镶上绿宝石,国际太空站拍下夜晚台湾，影片一上传，立刻受到热烈讨论，不少人都被台湾的美给吸引，?万家灯火构...,link,L
6747,7825,773000000000000,料理高校生,电视剧,【早鸟福利】\n方老师唯美的瞬间，就这样被我们捕捉到了 :P\n今天是11/11光棍节\n想...,动态时报相片,NaN,photo,L
6748,7826,773000000000000,料理高校生,电视剧,佩颖主任啊..\n您就别自暴自弃了~~\n \n莫非是摆脱烂桃花的新招?\n\n每周五晚间十...,动态时报相片,NaN,photo,L
6749,7827,242000000000000,ETtoday新闻云,新闻,快、合、作！（ Joe小姐）\n\n蔡依林 Jolin Tsai T.O.P. G-DRAGON,蔡依林、GD&T.O.P后台照曝光 甜呼：小爱心主人！ | 娱乐星光云 | ETtoday东...,「呸姊」蔡依林2日晚间出席南韩年度盛事MAMA舞台，她一开低胸紧身服装霸气登场，献唱舞曲《P...,link,L


In [40]:
counts50 = grp[(grp['H'] + grp['L'])>=50]
counts50['hl_score'] = counts50['H'] / (counts50['H'] + counts50['L'])
data_tmp = pd.merge(data_remove_bw, counts50, how='left', on='主页ID')
data_hl_score = data_tmp['hl_score'].fillna(0)
data_hl_score

<ipython-input-40-61051c25a3b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counts50['hl_score'] = counts50['H'] / (counts50['H'] + counts50['L'])


0       0.000000
1       0.000000
2       0.195730
3       0.000000
4       0.137514
          ...   
6746    0.063291
6747    0.064516
6748    0.064516
6749    0.195730
6750    0.039474
Name: hl_score, Length: 6751, dtype: float64

data_remove_bw['分类'] 提取 data_fl

In [26]:
data_remove_bw['分类'].value_counts()

新闻        4714
书籍杂志       812
节目         244
媒体其他       244
网路社群       232
电视频道       223
电视剧        199
电影公司        62
电影           8
广播           5
音乐           4
电影相关活动       4
Name: 分类, dtype: int64

In [42]:
# pay attentions to dataframe index
#TODO: add other categories methods.
column_mapping_str(data_remove_bw['分类'].unique())
fl_mapping = {'电视频道': 0, '网路社群': 1, '新闻': 2, '书籍杂志': 3, '电视剧': 4, '电影公司': 5, " \
             "'节目': 6, '媒体其他': 7, '电影': 5, '音乐': 7, '电影相关活动': 5, '广播': 7}
data_fl, fl_ohe = one_hot_encoder_column(data_remove_bw, '分类', fl_mapping, fill_na=7)
data_fl = pd.DataFrame(data_fl)
data_fl

,分类_0,分类_1,分类_2,分类_3,分类_4,分类_5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
6746,0.0,0.0,0.0,0.0,0.0,0.0
6747,0.0,0.0,0.0,1.0,0.0,0.0
6748,0.0,0.0,0.0,1.0,0.0,0.0
6749,0.0,1.0,0.0,0.0,0.0,0.0


## 发主类型处理
提取 data_doc_type <br>
不常见的类型归到video类 <br>
grp2 = data_remove_bw.pivot_table(index='发文类型', columns='y', values=['主页ID'], aggfunc='count').droplevel(0, axis=1)

In [43]:
doc_type_map = {'link': 1, 'photo': 2, 'video': 3}
data_doc_type, ohe_doc_type = one_hot_encoder_column(data_remove_bw, '发文类型', doc_type_map, fill_na=3)
data_doc_type


,发文类型_0,发文类型_1,发文类型_2
0,0.0,0.0,1.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,1.0,0.0,0.0
...,...,...,...
6746,1.0,0.0,0.0
6747,0.0,1.0,0.0
6748,0.0,1.0,0.0
6749,1.0,0.0,0.0


## 文本类型处理
data_remove_bw[~data_remove_bw['description'].isnull()]

In [49]:
stop_words = []
with open("./data/stopwords.txt", "r", encoding='gbk') as f_stopwords:
    for line in f_stopwords:
        line = line.replace("\r", "").replace("\n", "").strip()
        stop_words.append(line)
stop_words = set(stop_words)
print(len(stop_words))

2622


In [50]:
def split_word(document):
    import jieba
    import re
    if document is None or type(document) is float:
        return ""
    chinese_regular = u"([\u4e00-\u9fa5]+)"
    pattern = re.compile(chinese_regular)
    jieba.add_word("林智胜")
    jieba.add_word("郭严文")
    jieba.add_word("蔡依林")
    li = " ".join(jieba.cut(document))
    vocabulary = []
    chinese_only = pattern.findall(li)
    if chinese_only is not None and len(chinese_only) >= 1:

        for word in chinese_only:
            if word not in stop_words:
                vocabulary.append(word)
    return " ".join(vocabulary)

print(split_word("「呸姊」蔡依林2日晚间出席南韩年度盛事MAMA舞台，她一开低胸紧身服装霸气登场，献唱舞曲《P..."))

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/zz/8hl03_bd6sj_wrd8__z72r8r0000gn/T/jieba.cache
Loading model cost 0.894 seconds.
Prefix dict has been built successfully.


姊 蔡依林 晚间 出席 南韩 年度 盛事 舞台 一开 低胸 紧身 服装 霸气 登场 献唱 舞曲


In [54]:
# word-vector train using all data.
data['sp_msg'] = data['description'].apply(lambda doc: split_word(doc))
data[data['sp_msg'].map(lambda x: len(x) >= 2)]['sp_msg'].to_csv('lines.csv', index=False, header=False)


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
x = tfidf.fit_transform(data['sp_msg'])
df = pd.DataFrame(x.toarray(), columns=tfidf.get_feature_names())

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,一万,一万一千,一万元,一万多元,一上午,一上台,一上场,一下子,一下手,一丝,...,龙飞凤舞,龙首,龙骨,龚嫌,龚建嘉说,龚重安,龚重安说,龟山,龟山岛,龟甲
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from gensim.models import word2vec
sentences = word2vec.LineSentence("./lines.csv")
model = word2vec.Word2Vec(sentences, hs=1, min_count=5, window=3)

In [ ]:
from gensim.models import  doc2vec
model2 = doc2vec.Doc2Vec(sentences)
model2